# Premier league analysis from 2010/11 - 2019/20 seasons

In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/tara-nguyen/english-premier-league-datasets-for-10-seasons/main/epldat10seasons/epl-allseasons-matchstats.csv"
df = pd.read_csv(url, index_col=0)
print(df.head(5))

               Date        Referee     HomeTeam    AwayTeam FullTime Halftime  \
Season                                                                          
2010/11  2010-08-14         M Dean  Aston Villa    West Ham  HomeWin  HomeWin   
2010/11  2010-08-14         P Dowd    Blackburn     Everton  HomeWin  HomeWin   
2010/11  2010-08-14      S Attwell       Bolton      Fulham     Draw     Draw   
2010/11  2010-08-14  M Clattenburg      Chelsea   West Brom  HomeWin  HomeWin   
2010/11  2010-08-14       A Taylor   Sunderland  Birmingham     Draw  HomeWin   

         HomeGoals  HomeGoalsHalftime  HomeShots  HomeShotsOnTarget  ...  \
Season                                                               ...   
2010/11          3                  2         23                 11  ...   
2010/11          1                  1          7                  2  ...   
2010/11          0                  0         13                  9  ...   
2010/11          6                  2         18    